# 1. Libs

In [2]:
import requests
from bs4 import BeautifulSoup as bs

# 2. Pegando a URL e fazendo o requerimento

In [5]:
# Pegando a url
url = r'https://books.toscrape.com/'

# fazendo o requerimento:
response = requests.get(url)
# Status:
if response.status_code == 200:
    print("Sucesso")
else:
    print("Fail")

Sucesso


## 2.1 Parse e objeto soup

In [6]:
soup = bs(response.text,"html.parser")
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

# 3. web Scraping

# 3.1 Extraindo dados de uma pagina

In [16]:
# 3.1.2 Extraindo todas as informações da pagina com link e titulo:
soup.find_all('h3')

[<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>,
 <h3><a href="catalogue/tipping-the-velvet_999/index.html" title="Tipping the Velvet">Tipping the Velvet</a></h3>,
 <h3><a href="catalogue/soumission_998/index.html" title="Soumission">Soumission</a></h3>,
 <h3><a href="catalogue/sharp-objects_997/index.html" title="Sharp Objects">Sharp Objects</a></h3>,
 <h3><a href="catalogue/sapiens-a-brief-history-of-humankind_996/index.html" title="Sapiens: A Brief History of Humankind">Sapiens: A Brief History ...</a></h3>,
 <h3><a href="catalogue/the-requiem-red_995/index.html" title="The Requiem Red">The Requiem Red</a></h3>,
 <h3><a href="catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html" title="The Dirty Little Secrets of Getting Your Dream Job">The Dirty Little Secrets ...</a></h3>,
 <h3><a href="catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/ind

In [65]:
# Iremos extrair as informações de url a partir dessa parte:
pages = soup.find_all('h3')

In [66]:
# Obtendo a url
soup.find('a')['href']

'index.html'

Iremos acessar cada url para fazer o "scraping" de cada link

In [72]:
# Agora podemos começar a fazer a extração dos dados:

# lista com valores a qual iremos armazenar os dados:
dados = []

# Iterando pelos itens da lista:
for book in pages:
    book_url = book.find('a')['href']
    #print(book_url)
    url_book = url+book_url
    # Obtendo a url dos livros completa:
    #print(url_book)
    # usando o request para obter o objeto beutiful soup:
    bok_response = requests.get(url_book)
    # Obtendo o soup
    book_soup = bs(bok_response.content,"html.parser")
    #Verificando o soup dos livros:
    #print(book_soup)
    
    
    #Obtendo os elementos de cada livro:
    # 1. Titulo:
    titulo = book_soup.find('h1').get_text()
    categoria = book_soup.find('ul',class_="breadcrumb").find_all('a')[2].get_text().strip()
    avaliacao = book_soup.find('p',class_='star-rating')['class'][1]
    preco = book_soup.find('p',class_='price_color').get_text().strip()
    estoque = book_soup.find('p',class_='instock availability').get_text().strip()
    
    # Populando lista
    dados.append([titulo,categoria,avaliacao,preco,estoque])
    

In [68]:
dados

[['A Light in the Attic',
  'Poetry',
  'Three',
  '£51.77',
  'In stock (22 available)'],
 ['Tipping the Velvet',
  'Historical Fiction',
  'One',
  '£53.74',
  'In stock (20 available)'],
 ['Soumission', 'Fiction', 'One', '£50.10', 'In stock (20 available)'],
 ['Sharp Objects', 'Mystery', 'Four', '£47.82', 'In stock (20 available)'],
 ['Sapiens: A Brief History of Humankind',
  'History',
  'Five',
  '£54.23',
  'In stock (20 available)'],
 ['The Requiem Red',
  'Young Adult',
  'One',
  '£22.65',
  'In stock (19 available)'],
 ['The Dirty Little Secrets of Getting Your Dream Job',
  'Business',
  'Four',
  '£33.34',
  'In stock (19 available)'],
 ['The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
  'Default',
  'Three',
  '£17.93',
  'In stock (19 available)'],
 ['The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
  'Default',
  'Four',
  '£22.60',
  'In stock (19 available)'],
 ['The Black Maria'

# 3.2 Extraindo dados de todas as paginas

Basicamente o que iremos fazer é um loop entre todas as paginas desse site e assim obter um objeto soup de cada pagina, que posteriormente vai ser aplicado o mesmo looping de uma pagina, assim teremos os objetos soups de todos os libros do site

In [85]:
# dataframe total 
data_livros = []
# loop através das paginas:
for pag in range(1,31):
    # Pegando url:
    url_total = f'https://books.toscrape.com/catalogue/category/books_1/page-{pag}.html'
    # Requisicao e objeto soup:
    response = requests.get(url_total)
    soup_all = bs(response.content,"html.parser")
    # Notar que agora iremos extrair as informações da pagina desse objeto soup
    pages = soup_all.find_all('h3')
    
    
    # Agora iremos iteragir em cada um desses objetos soup de cada pagina especifica:
    for book in pages:
        book_url = book.find('a')['href']
        book_url = book_url[6:]
        url_book = r'https://books.toscrape.com/catalogue/'+book_url
        # Obtendo a url dos livros completa:
        # usando o request para obter o objeto beutiful soup:
        bok_response = requests.get(url_book)
        # Obtendo o soup
        book_soup = bs(bok_response.content,"html.parser")
        #Verificando o soup dos livros:
        #print(book_soup)


        #Obtendo os elementos de cada livro:
        # 1. Titulo:
        titulo = book_soup.find('h1').get_text()
        categoria = book_soup.find('ul',class_="breadcrumb").find_all('a')[2].get_text().strip()
        avaliacao = book_soup.find('p',class_='star-rating')['class'][1]
        preco = book_soup.find('p',class_='price_color').get_text().strip()
        estoque = book_soup.find('p',class_='instock availability').get_text().strip()

        # Populando lista
        data_livros.append([titulo,categoria,avaliacao,preco,estoque])
        



        
        
    
        
    
        
    
    

## 3.3 Criando dataframe

In [87]:
import pandas as pd

In [90]:
# Conversão para dataframe
df_livros = pd.DataFrame(data_livros,columns=['Titulo','Categoria','Avaliação','Preço','Estoque'])
# Display firsts 20 rows
df_livros.head(20)


,Titulo,Categoria,Avaliação,Preço,Estoque
0,A Light in the Attic,Poetry,Three,£51.77,In stock (22 available)
1,Tipping the Velvet,Historical Fiction,One,£53.74,In stock (20 available)
2,Soumission,Fiction,One,£50.10,In stock (20 available)
3,Sharp Objects,Mystery,Four,£47.82,In stock (20 available)
4,Sapiens: A Brief History of Humankind,History,Five,£54.23,In stock (20 available)
5,The Requiem Red,Young Adult,One,£22.65,In stock (19 available)
6,The Dirty Little Secrets of Getting Your Dream...,Business,Four,£33.34,In stock (19 available)
7,The Coming Woman: A Novel Based on the Life of...,Default,Three,£17.93,In stock (19 available)
8,The Boys in the Boat: Nine Americans and Their...,Default,Four,£22.60,In stock (19 available)
9,The Black Maria,Poetry,One,£52.15,In stock (19 available)


# 4. Tratamentos 

In [95]:
# Trabalhano no Preço:
df_livros['Preço'] = df_livros['Preço'].str[1:].astype(float)

In [106]:
# Estoque:
df_livros['Estoque']=df_livros['Estoque'].str[10:12].astype(int)

In [107]:
# verificando
df_livros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Titulo     400 non-null    object 
 1   Categoria  400 non-null    object 
 2   Avaliação  400 non-null    object 
 3   Preço      400 non-null    float64
 4   Estoque    400 non-null    int32  
dtypes: float64(1), int32(1), object(3)
memory usage: 14.2+ KB


In [108]:
df_livros.head()

,Titulo,Categoria,Avaliação,Preço,Estoque
0,A Light in the Attic,Poetry,Three,51.77,22
1,Tipping the Velvet,Historical Fiction,One,53.74,20
2,Soumission,Fiction,One,50.10,20
3,Sharp Objects,Mystery,Four,47.82,20
4,Sapiens: A Brief History of Humankind,History,Five,54.23,20


In [109]:
# Exports:
df_livros.to_excel('livros totais.xlsx')
